# Business and data understanding

## Purpose
This notebook contains the data preparation phase for the .

## Tasks
- [ ] Create a baseline model.
- [ ] Submit the test result.

# Setup

## Library import

In [1]:
from datetime import datetime 
import json
import os
from pathlib import Path

from feature_engine.encoding import CountFrequencyEncoder, RareLabelEncoder, OneHotEncoder
import folium
import humps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report, log_loss
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB, CategoricalNB, GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, NuSVC
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as ply

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

# Options for pandas and plotly
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 200
plotly.offline.init_notebook_mode(connected=True)

## Local library import

In [2]:
if Path.cwd().name == 'notebooks':
    os.chdir('../')

from src.utils.data_describe import serie_nulos, cardinalidade
from src.data.data_preprocessing import (
    TransformCordinates, address_split, create_date_based_columns, snake_case_columns,
    create_simplified_address_column, one_hot_encoding_target
)

if Path.cwd().name == 'sf_crime':
    os.chdir('./notebooks/')

## Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [3]:
RAW_DATA = '../data/raw/' 
EXTERNAL_DATA = '../data/external/' 
INTERIM_DATA = '../data/interim/' 
PROCESSED_DATA = '../data/processed/'
REFERENCES = '../references/'
RANDOM_STATE = 42

lst_columns = [
        'ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY', 'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
        'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION', 'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING',
        'FRAUD', 'GAMBLING', 'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING', 'MISSING PERSON',
        'NON-CRIMINAL', 'OTHER OFFENSES', 'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE', 'ROBBERY',
        'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE', 'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY',
        'SUICIDE', 'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT', 'WARRANTS', 'WEAPON LAWS'
    ]

## Data import
We retrieve all the required data for the analysis.

In [4]:
try:
    X_train = pd.read_parquet(INTERIM_DATA + 'X_train_transformed.pqt')
    X_validation = pd.read_parquet(INTERIM_DATA + 'X_validation_transformed.pqt')
    X_kaggle_test = pd.read_parquet(INTERIM_DATA + 'X_kaggle_test_transformed_transformed.pqt')
    y_train_ohe = pd.read_parquet(INTERIM_DATA + 'y_train_ohe.pqt')
    y_validation_ohe = pd.read_parquet(INTERIM_DATA + 'y_validation_ohe.pqt')
    y_train_ordinal = pd.read_parquet(INTERIM_DATA + 'y_train_ordinal.pqt')
    y_validation_ordinal = pd.read_parquet(INTERIM_DATA + 'y_validation_ordinal.pqt')
    print('Parquet files loaded.')

except FileNotFoundError as e:
    print('Files were not found.')
    
print(f"""
X_train: {X_train.shape}
X_validation: {X_validation.shape}
X_kaggle_test: {X_kaggle_test.shape}

y_train_ohe: {y_train_ohe.shape}
y_validation_ohe: {y_validation_ohe.shape}

y_train_ordinal: {y_train_ordinal.shape}
y_validation_ordinal: {y_validation_ordinal.shape}
""")

X_train.tail(3)

Parquet files loaded.

X_train: (790244, 10)
X_validation: (87805, 10)
X_kaggle_test: (884262, 10)

y_train_ohe: (790244, 39)
y_validation_ohe: (87805, 39)

y_train_ordinal: (790244, 1)
y_validation_ordinal: (87805, 1)



,dates_year,dates_month,dates_hour,dates_day,is_daytime,day_of_week,pd_district,x,y,simplified_address
131932,2013,8,22,10,0,0.144286,0.136463,-122.426956,37.769247,0.039924
671155,2005,11,5,4,0,0.152310,0.102038,-122.386942,37.754168,0.776874
121958,2013,9,12,30,1,0.138624,0.179080,-122.408068,37.783992,0.039924


# 1st experiment: Multinomial Naive Bayes (baseline)

- Naive bayes. One model for each crime category.

In [ ]:
X_train_mod = X_train.copy()
X_train_mod['x'] = X_train_mod['x'].abs()

model = MultinomialNB()
model.fit(X_train_mod, y_train_ordinal.values[:, 0])

y_pred = model.predict(X_validation)

In [ ]:
# print(log_loss(y_validation_ordinal, y_pred, labels=np.arange(0,38).tolist()))

print(classification_report(y_validation_ordinal, y_pred, zero_division=0))

## Kaggle's test dataset

In [ ]:
# Predicting the probabilities from X_kaggle_test.
y_kaggle_pred = model.predict_proba(X_kaggle_test).round(2)
y_kaggle_pred = pd.DataFrame(y_kaggle_pred, columns=lst_columns).reset_index().rename(columns={'index': 'Id'})
y_kaggle_pred.head(3)

In [ ]:
y_kaggle_pred.to_csv(PROCESSED_DATA + 'y_kaggle_pred_1st_prep_1st_model.zip', sep=',', index=False, compression='zip')

### Kaggle's score:

Score: 3.70134

# 2nd experiment: Random Forest with default parameters

In [ ]:
model = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=2)
model.fit(X_train, y_train_ordinal.values[:, 0])

y_pred = model.predict(X_validation)

print(classification_report(y_validation_ordinal, y_pred, zero_division=0))

## Kaggle's test dataset

In [ ]:
# Predicting the probabilities from X_kaggle_test.
y_kaggle_pred = model.predict_proba(X_kaggle_test).round(2)
y_kaggle_pred = pd.DataFrame(y_kaggle_pred, columns=lst_columns).reset_index().rename(columns={'index': 'Id'})
y_kaggle_pred.head(3)

In [ ]:
y_kaggle_pred.to_csv(PROCESSED_DATA + 'y_kaggle_pred_1st_prep_2nd_model.zip', sep=',', index=False, compression='zip')

# 3rd experiment: Multinomial Naive Bayes (baseline) with cyclical features after sine/cosine transformation

## Data import

In [ ]:
try:
    X_train = pd.read_parquet(INTERIM_DATA + 'X_train_transformed_2nd_dataprep.pqt')
    X_validation = pd.read_parquet(INTERIM_DATA + 'X_validation_transformed_2nd_dataprep.pqt')
    X_kaggle_test = pd.read_parquet(INTERIM_DATA + 'X_kaggle_test_transformed_transformed_2nd_dataprep.pqt')

    y_train_ordinal = pd.read_parquet(INTERIM_DATA + 'y_train_ordinal.pqt')
    y_validation_ordinal = pd.read_parquet(INTERIM_DATA + 'y_validation_ordinal.pqt')
    print('Parquet files loaded.')

except FileNotFoundError as e:
    print('Files were not found.')
    
print(f"""
X_train: {X_train.shape}
X_validation: {X_validation.shape}
X_kaggle_test: {X_kaggle_test.shape}

y_train_ordinal: {y_train_ordinal.shape}
y_validation_ordinal: {y_validation_ordinal.shape}
""")

X_train.tail(3)

In [ ]:
X_train_mod = X_train.copy()

for column in ['x', 'dates_month_sin', 'dates_month_cos', 'dates_day_sin', 'dates_day_cos', 'dates_hour_sin', 'dates_hour_cos']:
    X_train_mod[column] = X_train_mod[column].abs()

model = MultinomialNB()
model.fit(X_train_mod, y_train_ordinal.values[:, 0])

y_pred = model.predict(X_validation)

In [ ]:
# print(log_loss(y_validation_ordinal, y_pred, labels=np.arange(0,38).tolist()))

print(classification_report(y_validation_ordinal, y_pred, zero_division=0))

## Kaggle's test dataset

In [ ]:
# Predicting the probabilities from X_kaggle_test.
y_kaggle_pred = model.predict_proba(X_kaggle_test).round(2)
y_kaggle_pred = pd.DataFrame(y_kaggle_pred, columns=lst_columns).reset_index().rename(columns={'index': 'Id'})
y_kaggle_pred.head(3)

In [ ]:
y_kaggle_pred.to_csv(PROCESSED_DATA + 'y_kaggle_pred_2nd_prep_3rd_model.zip', sep=',', index=False, compression='zip')

### Kaggle's score:

Score: 3.44058 (7% better (less) than baseline)

# 4th experiment: Complement Naive Bayes with cyclical features after sine/cosine transformation

## Data import

In [ ]:
try:
    X_train = pd.read_parquet(INTERIM_DATA + 'X_train_transformed_2nd_dataprep.pqt')
    X_validation = pd.read_parquet(INTERIM_DATA + 'X_validation_transformed_2nd_dataprep.pqt')
    X_kaggle_test = pd.read_parquet(INTERIM_DATA + 'X_kaggle_test_transformed_transformed_2nd_dataprep.pqt')

    y_train_ordinal = pd.read_parquet(INTERIM_DATA + 'y_train_ordinal.pqt')
    y_validation_ordinal = pd.read_parquet(INTERIM_DATA + 'y_validation_ordinal.pqt')
    print('Parquet files loaded.')

except FileNotFoundError as e:
    print('Files were not found.')
    
print(f"""
X_train: {X_train.shape}
X_validation: {X_validation.shape}
X_kaggle_test: {X_kaggle_test.shape}

y_train_ordinal: {y_train_ordinal.shape}
y_validation_ordinal: {y_validation_ordinal.shape}
""")

X_train.tail(3)

In [ ]:
X_train_mod = X_train.copy()

for column in ['x', 'dates_month_sin', 'dates_month_cos', 'dates_day_sin', 'dates_day_cos', 'dates_hour_sin', 'dates_hour_cos']:
    X_train_mod[column] = X_train_mod[column].abs()

model = ComplementNB()
model.fit(X_train_mod, y_train_ordinal.values[:, 0])

y_pred = model.predict(X_validation)

In [ ]:
# print(log_loss(y_validation_ordinal, y_pred, labels=np.arange(0,38).tolist()))

print(classification_report(y_validation_ordinal, y_pred, zero_division=0))

## Kaggle's test dataset

In [ ]:
# Predicting the probabilities from X_kaggle_test.
y_kaggle_pred = model.predict_proba(X_kaggle_test).round(2)
y_kaggle_pred = pd.DataFrame(y_kaggle_pred, columns=lst_columns).reset_index().rename(columns={'index': 'Id'})
y_kaggle_pred.head(3)

In [ ]:
y_kaggle_pred.to_csv(PROCESSED_DATA + 'y_kaggle_pred_2nd_prep_4rd_model.zip', sep=',', index=False, compression='zip')

### Kaggle's score:

Score: 3.66351 (1% better (less) than baseline)

# 5th experiment: Complement Naive Bayes with cyclical features after sine/cosine transformation

## Data import

In [ ]:
try:
    X_train = pd.read_parquet(INTERIM_DATA + 'X_train_transformed_2nd_dataprep.pqt')
    X_validation = pd.read_parquet(INTERIM_DATA + 'X_validation_transformed_2nd_dataprep.pqt')
    X_kaggle_test = pd.read_parquet(INTERIM_DATA + 'X_kaggle_test_transformed_transformed_2nd_dataprep.pqt')

    y_train_ordinal = pd.read_parquet(INTERIM_DATA + 'y_train_ordinal.pqt')
    y_validation_ordinal = pd.read_parquet(INTERIM_DATA + 'y_validation_ordinal.pqt')
    print('Parquet files loaded.')

except FileNotFoundError as e:
    print('Files were not found.')
    
print(f"""
X_train: {X_train.shape}
X_validation: {X_validation.shape}
X_kaggle_test: {X_kaggle_test.shape}

y_train_ordinal: {y_train_ordinal.shape}
y_validation_ordinal: {y_validation_ordinal.shape}
""")

X_train.tail(3)

In [ ]:
X_train_mod = X_train.copy()
X_validation_mod = X_validation.copy()
X_kaggle_test_mod = X_kaggle_test.copy()

for column in ['x', 'dates_month_sin', 'dates_month_cos', 'dates_day_sin', 'dates_day_cos', 'dates_hour_sin', 'dates_hour_cos']:
    X_train_mod[column] = X_train_mod[column].abs()
    X_validation_mod[column] = X_validation_mod[column].abs()
    X_kaggle_test_mod[column] = X_kaggle_test_mod[column].abs()
    

model = CategoricalNB()
model.fit(X_train_mod, y_train_ordinal.values[:, 0])

y_pred = model.predict(X_validation_mod)

In [ ]:
# print(log_loss(y_validation_ordinal, y_pred, labels=np.arange(0,38).tolist()))

print(classification_report(y_validation_ordinal, y_pred, zero_division=0))

## Kaggle's test dataset

In [ ]:
# Predicting the probabilities from X_kaggle_test.
y_kaggle_pred = model.predict_proba(X_kaggle_test_mod).round(2)
y_kaggle_pred = pd.DataFrame(y_kaggle_pred, columns=lst_columns).reset_index().rename(columns={'index': 'Id'})
y_kaggle_pred.head(3)

In [ ]:
y_kaggle_pred.to_csv(PROCESSED_DATA + 'y_kaggle_pred_2nd_prep_5th_model.zip', sep=',', index=False, compression='zip')

### Kaggle's score:

Score: 3.73690 (1% worse (higher) than baseline)

# 6th experiment: Gaussian Naive Bayes with cyclical features after sine/cosine transformation

## Data import

In [ ]:
try:
    X_train = pd.read_parquet(INTERIM_DATA + 'X_train_transformed_2nd_dataprep.pqt')
    X_validation = pd.read_parquet(INTERIM_DATA + 'X_validation_transformed_2nd_dataprep.pqt')
    X_kaggle_test = pd.read_parquet(INTERIM_DATA + 'X_kaggle_test_transformed_transformed_2nd_dataprep.pqt')

    y_train_ordinal = pd.read_parquet(INTERIM_DATA + 'y_train_ordinal.pqt')
    y_validation_ordinal = pd.read_parquet(INTERIM_DATA + 'y_validation_ordinal.pqt')
    print('Parquet files loaded.')

except FileNotFoundError as e:
    print('Files were not found.')
    
print(f"""
X_train: {X_train.shape}
X_validation: {X_validation.shape}
X_kaggle_test: {X_kaggle_test.shape}

y_train_ordinal: {y_train_ordinal.shape}
y_validation_ordinal: {y_validation_ordinal.shape}
""")

X_train.tail(3)

In [ ]:
X_train_mod = X_train.copy()
X_validation_mod = X_validation.copy()
X_kaggle_test_mod = X_kaggle_test.copy()

for column in ['x', 'dates_month_sin', 'dates_month_cos', 'dates_day_sin', 'dates_day_cos', 'dates_hour_sin', 'dates_hour_cos']:
    X_train_mod[column] = X_train_mod[column].abs()
    X_validation_mod[column] = X_validation_mod[column].abs()
    X_kaggle_test_mod[column] = X_kaggle_test_mod[column].abs()
    

model = GaussianNB()
model.fit(X_train_mod, y_train_ordinal.values[:, 0])

y_pred = model.predict(X_validation_mod)

In [ ]:
# print(log_loss(y_validation_ordinal, y_pred, labels=np.arange(0,38).tolist()))

print(classification_report(y_validation_ordinal, y_pred, zero_division=0))

## Kaggle's test dataset

In [ ]:
# Predicting the probabilities from X_kaggle_test.
y_kaggle_pred = model.predict_proba(X_kaggle_test_mod).round(2)
y_kaggle_pred = pd.DataFrame(y_kaggle_pred, columns=lst_columns).reset_index().rename(columns={'index': 'Id'})
y_kaggle_pred.head(3)

In [ ]:
y_kaggle_pred.to_csv(PROCESSED_DATA + 'y_kaggle_pred_2nd_prep_6th_model.zip', sep=',', index=False, compression='zip')

### Kaggle's score:

Score: 3.44058 (7% better (less) than baseline)

# 7th experiment: Multinomial NB with cyclical features after sine/cosine transformation

## Data import

In [12]:
try:
    X_train = pd.read_parquet(INTERIM_DATA + 'X_train_transformed_3rd_dataprep.pqt')
    X_validation = pd.read_parquet(INTERIM_DATA + 'X_validation_transformed_3rd_dataprep.pqt')
    X_kaggle_test = pd.read_parquet(INTERIM_DATA + 'X_kaggle_test_transformed_transformed_3rd_dataprep.pqt')

    y_train_ordinal = pd.read_parquet(INTERIM_DATA + 'y_train_ordinal.pqt')
    y_validation_ordinal = pd.read_parquet(INTERIM_DATA + 'y_validation_ordinal.pqt')
    print('Parquet files loaded.')

except FileNotFoundError as e:
    print('Files were not found.')
    
print(f"""
X_train: {X_train.shape}
X_validation: {X_validation.shape}
X_kaggle_test: {X_kaggle_test.shape}

y_train_ordinal: {y_train_ordinal.shape}
y_validation_ordinal: {y_validation_ordinal.shape}
""")

X_train.tail(3)

Parquet files loaded.

X_train: (790244, 13)
X_validation: (87805, 13)
X_kaggle_test: (884262, 13)

y_train_ordinal: (790244, 1)
y_validation_ordinal: (87805, 1)



,dates_year,is_daytime,day_of_week,pd_district,x,y,simplified_address,dates_month_sin,dates_month_cos,dates_day_sin,dates_day_cos,dates_hour_sin,dates_hour_cos
131932,2013,0,0.144286,0.136463,-122.426956,37.769247,0.039924,0.066987,0.250000,0.948869,0.277951,0.364361,0.981372
671155,2005,0,0.152310,0.102038,-122.386942,37.754168,0.776874,0.250000,0.933013,0.862308,0.844083,0.989530,0.599865
121958,2013,1,0.138624,0.179080,-122.408068,37.783992,0.039924,0.000000,0.500000,0.398965,0.989739,0.431254,0.000000


In [21]:
X_train_mod = X_train.copy()
X_validation_mod = X_validation.copy()
X_kaggle_test_mod = X_kaggle_test.copy()

for column in ['x', 'y']:
    X_train_mod[column] = X_train_mod[column].abs()
    X_validation_mod[column] = X_validation_mod[column].abs()
    X_kaggle_test_mod[column] = X_kaggle_test_mod[column].abs()

model = MultinomialNB()
model.fit(X_train_mod, y_train_ordinal.values[:, 0])

y_pred = model.predict(X_validation_mod)

In [22]:
# print(log_loss(y_validation_ordinal, y_pred, labels=np.arange(0,38).tolist()))

print(classification_report(y_validation_ordinal, y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       168
           1       0.00      0.00      0.00      7632
           2       0.00      0.00      0.00        41
           3       0.00      0.00      0.00        28
           4       0.00      0.00      0.00      3762
           5       0.00      0.00      0.00       409
           6       0.00      0.00      0.00       212
           7       0.00      0.00      0.00      5437
           8       0.00      0.00      0.00       421
           9       0.00      0.00      0.00        91
          10       0.00      0.00      0.00        29
          11       0.00      0.00      0.00        44
          12       0.00      0.00      0.00      1043
          13       0.00      0.00      0.00      1621
          14       0.00      0.00      0.00        14
          15       0.00      0.00      0.00       232
          16       0.20      1.00      0.33     17455
          17       0.00    

## Kaggle's test dataset

In [23]:
# Predicting the probabilities from X_kaggle_test.
y_kaggle_pred = model.predict_proba(X_kaggle_test_mod).round(2)
y_kaggle_pred = pd.DataFrame(y_kaggle_pred, columns=lst_columns).reset_index().rename(columns={'index': 'Id'})
y_kaggle_pred.head(3)

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,EXTORTION,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PORNOGRAPHY/OBSCENE MAT,PROSTITUTION,RECOVERED VEHICLE,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.0,0.09,0.0,0.0,0.04,0.01,0.0,0.05,0.01,0.0,0.0,0.0,0.01,0.02,0.0,0.0,0.20,0.0,0.0,0.02,0.09,0.14,0.0,0.01,0.0,0.03,0.0,0.01,0.01,0.0,0.01,0.0,0.03,0.0,0.01,0.06,0.07,0.04,0.01
1,1,0.0,0.09,0.0,0.0,0.04,0.01,0.0,0.06,0.01,0.0,0.0,0.0,0.01,0.02,0.0,0.0,0.19,0.0,0.0,0.02,0.09,0.14,0.0,0.01,0.0,0.03,0.0,0.01,0.01,0.0,0.01,0.0,0.03,0.0,0.01,0.06,0.06,0.05,0.01
2,2,0.0,0.09,0.0,0.0,0.04,0.01,0.0,0.05,0.01,0.0,0.0,0.0,0.01,0.02,0.0,0.0,0.20,0.0,0.0,0.02,0.09,0.14,0.0,0.01,0.0,0.03,0.0,0.01,0.01,0.0,0.01,0.0,0.03,0.0,0.01,0.06,0.07,0.04,0.01


In [24]:
y_kaggle_pred.to_csv(PROCESSED_DATA + 'y_kaggle_pred_3rd_prep_7th_model.zip', sep=',', index=False, compression='zip')

### Kaggle's score:

Score: 3.42403 (8% better (less) than baseline)